In [1]:
import requests
import bs4
import datetime
import re  
import json

# geting the data from the web 
1. posts: 
    1.  text
    1. engagment data 
    1. flairs 
    1. poster
1. time

In [2]:
def isscroler(css_class):
    return css_class is not None and "scrollerItem" in css_class

def get_d(b):
    l=b.find_all("html")[0].find_all("script",{"id":"data"})[0]
    #isolate the dictionary.
    jsonStr = re.search(r'\{.*\}', str(l)).group()
    # Create dictionary
    return json.loads(jsonStr)["posts"]["models"] 

def get_info(b):
    d=get_d(b)
    scrolers=b.find_all(class_=isscroler) 
    ans=[]
    for x in scrolers:
        if("Post" not in x.attrs["class"]):
            print("skiped")
            continue
        base=x.find_all(role="screen-reader")
        comments=["href" in x.parent.parent.parent.attrs.keys() for x in base]
        base=[x.text for x in base]
        base=list(zip(base,comments))
        base.extend(["missing entry" for _ in range(2-len(base))])
        
        meta=d[x.attrs["id"]]
        #if ("source" in meta.keys()):
        for k in ["numComments","upvoteRatio","score","authorId","title"]:
            base.append(meta[k])
        
        base.append([f["text"] for f in meta["flair"]])
            
        try: 
            base.append(d[x.attrs["id"]]["media"]["richtextContent"]["document"])
        except:
            base.append("empty")
        ans.append(base)
    return ans

START="https://web.archive.org/web/"
def get_time(raw):
    if(raw.url[:28]!=START):
        raise("you are passing the wrong url")
    s=raw.url[28:].split("/")[0]
    year=int(s[0:4])
    month=int(s[4:6])
    day=int(s[6:8])
    hour=int(s[8:10])
    minute=int(s[10:12])
    second=int(s[12:14])
    return datetime.datetime(year=year,month=month,day=day,hour=hour,minute=minute,second=second)

In [85]:
def mine(url):
    r=requests.get(url)
    r.close()
    b=bs4.BeautifulSoup(r.content)
    return (get_info(b),get_time(r))
#mined=mine("https://web.archive.org/web/20210103/https://www.reddit.com/r/WhatCarShouldIBuy/")
mined=mine("https://web.archive.org/web/20210104005418/https://www.reddit.com/r/cars/")
mined

skiped


([['missing entry',
   'missing entry',
   243,
   0.88,
   31,
   't2_6l4z3',
   'Weekly - What Car Should I Buy Megathread',
   [],
   [{'c': [{'e': 'raw', 't': 'Weekly - What Car Should I Buy Megathread'}],
     'e': 'h',
     'l': 1},
    {'e': 'hr'},
    {'c': [{'e': 'text',
       't': 'Any posts pertaining to car buying suggestions or advice go in this weekly Megathread. A fresh thread will be posted every Monday and posts auto sorted by new. A few other subreddits worth checking out that will help your car buying expierance is '},
      {'e': 'r/', 't': 'WhatCarShouldIBuy', 'l': True},
      {'e': 'text', 't': ', '},
      {'e': 'r/', 't': 'UsedCars', 'l': True},
      {'e': 'text', 't': ' and '},
      {'e': 'r/', 't': 'AskCarSales', 'l': True},
      {'e': 'text',
       't': '. Make/Model specific questions should be asked on Make/Model specific subreddits. Check the '},
      {'e': 'link',
       'u': 'https://web.archive.org/web/20210104005418/https://www.reddit.com/r/Auto

In [86]:
ls=[len(x) for x in mined[0]]
LENGTH=max(ls)
ls

[9, 9, 9, 9, 9]

In [131]:
#mined[0][-1][-1]
#[[(y,i) for y in x[-1] if "c" not in y.keys()] for i,x in enumerate(mined[0]) if type(x[-1][0])==dict]
[[[(k,i,x.keys()) for x in y["c"] if "t" not in x.keys()] for i,y in enumerate(exmple[-1]) if y["e"] != "hr"] for k,exmple in enumerate(mined[0]) if exmple[-1] !="empty"]

[[[],
  [],
  [],
  [],
  [(0, 6, dict_keys(['c', 'e', 'o']))],
  [(0, 7, dict_keys(['c', 'e', 'o']))],
  [(0, 8, dict_keys(['c', 'e', 'o']))],
  [(0, 9, dict_keys(['c', 'e', 'o']))]],
 [[]]]

In [134]:
#print( mined[0][0][-3])
x=mined[0][0][-1]
#l=[x["c"] for x in x]
#[[x["t"] for x in y["c"]] for y in x]
x[7]

{'c': [{'c': [{'c': [{'c': [{'e': 'text',
         't': 'Price range: (Minimum-Maximum in your local currency)',
         'f': [[1, 0, 12]]}],
       'e': 'par'}],
     'e': 'li'}],
   'e': 'list',
   'o': False}],
 'e': 'blockquote'}

In [120]:
def stringit(l):
    s=""
    for x in l:
        s+=x
    return s+"\n"

def extract_text(x):
    if (x=="empty"):
        return ""
    x= [[x["t"] for x in y["c"]] for y in x if y["e"] != "hr"]#"c" in y.keys()] 
    x =[stringit(x) for x in x]
    return stringit(x)
s=stringit([extract_text(x[-1])+"\n___________\n\n" for x in mined[0]])
print(s)

KeyError: 't'

In [94]:
[x[-2] for x in mined[0]]

[[], [], ['video'], ['video'], ['video']]

# issues found 
testing at https://web.archive.org/web/20210103/https://www.reddit.com/r/WhatCarShouldIBuy/

1. first post dosent fetch comments and upvotes 
2. fetched comments dont much the dictonary comments

In [ ]:
r=requests.get("https://web.archive.org/web/20210103/https://www.reddit.com/r/WhatCarShouldIBuy/")
r.close()
b=bs4.BeautifulSoup(r.content)
scrolers=b.find_all(class_=isscroler)

In [ ]:
d=get_d(b)
x=d["t3_ifhic8"]
x1=d["t3_kl86w6"]

[k for k in x.keys() if k not in x1.keys()]

In [ ]:
x["source"]

In [ ]:
x

In [13]:
import striprtf

In [16]:
striprtf.striprtf.PATTERN

re.compile(r"\\([a-z]{1,32})(-?\d{1,10})?[ ]?|\\'([0-9a-f]{2})|\\([^a-z])|([{}])|[\r\n]+|(.)",
re.IGNORECASE|re.UNICODE)